<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/temperatura_max_mensile_capoluoghi-regione_1.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [5]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests

#ee.Authenticate()
ee.Initialize()
!pip install rasterstats
!pip install wget
import wget
from rasterstats import zonal_stats

In [6]:
anno = 2023
mesi = [6,7,8,9]

In [7]:
import folium
from folium import plugins

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [8]:
def crea_mappa(nomecom,periodo):
  m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
  folium.Choropleth(
      geo_data=df_sez11,
      name="Media per sezione delle temperature massime mensili",
      data=sdf1,
      columns=["SEZ", "m202308"],
      key_on="feature.properties.SEZ",
      fill_color= 'YlOrRd', #YlGn
      fill_opacity=0.7,
      line_opacity=0.2,
      legend_name="Unemployment Rate (%)",
  ).add_to(m)

  folium.LayerControl().add_to(m)
  m.save(comune + '_' + str(anno) + mese1 + '.html')

In [9]:
def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 50,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

In [10]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [11]:
# carica lo shape dei capoluoghi di regione
nomecap = 'comprovll4.geojson'
if not(os.path.isfile(nomecap)):
  #wget.download('https://github.com/cimbelli/Surface-Heat-Islands/raw/5006ef2e37091eedf421f65ed9cbf80c15ccbdfd/' + nomecap)
  wget.download('https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/' + nomecap)
g = open(nomecap)
cc = json.load(g)

gdf = gpd.read_file(nomecap)
centro = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs)
#gdf["lon"] = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs) #gdf["geometry"].to_crs(4326).centroid.x
#gdf["lat"] = gdf["geometry"].centroid.y
gdf["lon"] = centro.x
gdf["lat"] = centro.y

In [24]:
 print((6-len(str(id1)))*'0'+ str(id1))

071024


In [ ]:
try:
  del sdf
except:
  pass
for i in range(len(cc['features'])):

    reg = cc['features'][i]['properties']['COD_REG']
    reg1 = ('0' if reg < 10 else '') + str(reg)
    id1 = cc['features'][i]['properties']['PRO_COM']
    procom1 = '0' * (6-len(str(id1))) + str(id1)
    print((6-len(str(id1)))*'0', procom1)

    coords = cc['features'][i]['geometry']['coordinates']
    fc = ee.FeatureCollection(cc['features'])
    comune = cc['features'][i]['properties']['COMUNE']

    nomefilesez = 'sez11_' + procom1 +'.geojson'
    if not(os.path.isfile(nomefilesez)):
      urlfilesez = 'https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/sez11/' + nomefilesez
      wget.download(urlfilesez)
    g1 = open(nomefilesez)
    df_sez11 = gpd.read_file(g1)
    df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
    df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)

    lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
    lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

    minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
    maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
    miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
    maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

    region = ee.Geometry.BBox(minx, miny, maxx, maxy)

    for mese in mesi:
        nome_img = 'ST_' + comune + '_'+ str(anno) + '0' + str(mese) + '.tif'
        mese2 = ('0' + str(mese+1)) if mese < 9 else str(mese+1)

        if not(os.path.isfile(nome_img)):

            L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-0' + str(mese) +'-01', str(anno) + '-' + mese2 +'-01')
            bands = ['ST_B10', 'QA_PIXEL']
            bandName = ['ST', 'QA_PIXEL']
            filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 20)).select(bands, bandName)#.map(cloudMask)
            LandsatColl = filtered_L8.merge(filtered_L9)
            LandsatColl = LandsatColl.select('ST').filterBounds(region)

            lst = LandsatColl.max().clip(fc)
            thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)


            # zonal stats per sezioni
            print('scarico immagine', comune, anno, "mese",mese, "n° acquisizioni", LandsatColl.size().getInfo())

            scarica(thermal, nome_img, region)
        else:
            print(nome_img, 'già presente')

 # se è il primo mese usa il df delle sezioni originario altrimenti quello con precedenti stats
        try:
            df_o = sdf
            #print("sdf esistente")
        except:
            df_o = df_sez11
            #stats = zonal_stats(df_o, nome_img, stats="mean")

            #print("sdf non esistente")
        stats = zonal_stats(df_o, nome_img, stats="mean")
        sdf = pd.DataFrame(stats)
        sdf.rename(columns={'mean': 'm' + str(anno) + '0' + str(mese) }, inplace=True)
        sdf = pd.concat([df_o, sdf], axis=1)
        stats.clear()

        #print('regione', reg, comune, 'anno', anno, 'mese', mese)#, center)#, aoi)
    sdf1 = sdf.drop('geometry', axis=1) # sdf.drop('geometry', inplace=True, axis=1)
    sdf1.to_excel(comune + '_' + 'stats.xlsx', index=False)
    sdf1.to_csv(comune + '_' + 'stats.csv', index=False)

    #del sdf
    print("Create statistiche mensili per", comune, "estate", anno)

    # Crea la mappa per il comune e il mese di agosto
    m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
    folium.Choropleth(
        geo_data=df_sez11,
        name ="Media per sezione delle temperature massime mensili - " + comune + '-' + 'Agosto',
        data=sdf1,
        columns=["SEZ", "m202308"],
        key_on="feature.properties.SEZ",
        fill_color= 'YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Media per sezione delle temperature massime mensili",
    ).add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
    SuburbName = folium.features.GeoJson(sdf, style_function=style_function,
        control=False, tooltip=folium.features.GeoJsonTooltip(
            fields=['SEZ',"m202308"],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
        )
    )
    m.add_child(SuburbName)
    m.keep_in_front(SuburbName)
    basemaps['Google Satellite'].add_to(m)

    folium.LayerControl().add_to(m)
    m.save(comune + '_' + str(anno) + '08' + '.html')
    print("Creata mappa interattiva per", comune, "agosto", anno)

    break


0 071024
scarico immagine Foggia 2023 mese 6 n° acquisizioni 3
scarico immagine Foggia 2023 mese 7 n° acquisizioni 7
scarico immagine Foggia 2023 mese 8 n° acquisizioni 12
scarico immagine Foggia 2023 mese 9 n° acquisizioni 7
Create statistiche mensili per Foggia estate 2023


In [ ]:
sdf1

In [ ]:
# Crea la mappa per il comune e il mese
m = folium.Map(location=(centro.y[0],centro.x[0] ), tiles="cartodb positron", zoom_start=12)
folium.Choropleth(
    geo_data=df_sez11,
    name="Media per sezione delle temperature massime mensili",
    data=sdf1,
    columns=["SEZ", "m202308"],
    key_on="feature.properties.SEZ",
    fill_color= 'YlOrRd', #YlGn
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Media per sezione delle temperature massime mensili",
).add_to(m)

style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0.1,
                            'weight': 0.1}
SuburbName = folium.features.GeoJson(
    sdf,
    style_function=style_function,
    control=False,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SEZ',"m202308"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
m.add_child(SuburbName)
m.keep_in_front(SuburbName)


basemaps['Google Satellite'].add_to(m)
basemaps['Google Terrain'].add_to(m)
folium.LayerControl().add_to(m)
#m.save(comune + '_' + str(anno) + str(mese) + '.html')
m

In [ ]:
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0.1,
                            'weight': 0.1}
SuburbName = folium.features.GeoJson(
    df_sez11,
    style_function=style_function,
    control=False,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SEZ',"m202308"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
m.add_child(SuburbName)
m.keep_in_front(SuburbName)